In [1]:
# Import modules
# ipython magic to plot in line
%matplotlib inline
#import mpld3
#mpld3.enable_notebook()
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from astropy.io import ascii
import pytz
# OS interaction
import sys
import os

In [2]:
# Directories
# Path to raw data
main_dir   = os.path.normpath(r'F:\Work\e\Data\Obs\Canada_Project_Sites\Nov_2014_snow_storm_data')
# Ascii input folder
dir_in     = main_dir + '\QC_ascii'
# netcdf output folder
dir_out    = main_dir + '\\QC_netcdf'

In [3]:
# Define input format of ascii files
input_format = 'CRHO_TELM'

In [4]:
if input_format == 'CRHO_TELM':
    # Ascii data format info
    c_header = 4 # Header lines
    c_column_line = 1 # line where column names start
    c_delimiter = ','
    # time zone variables
    #tz_in = pytz.timezone('Canada/Mountain')
    tz_in = pytz.timezone('Etc/GMT-6')

In [5]:
# Get file in info
os.chdir(dir_in) # Move to input
content = os.listdir(os.getcwd()) # Get list of files
num_files = len([name for name in os.listdir('.') if os.path.isfile(name)]) # Get number of files in

In [6]:
#dat = ascii.read(cfile,header_start=c_column_line,data_start=c_header,delimiter=c_delimiter,exclude_names='N/A',guess=False)

In [7]:
# Read in each file
ds_ALL = []
for cfile in content:
    # Get current station name
    csta_name = cfile[0:3] # Take the first three letter abbreviation as the name
    print('Processing ' + csta_name)
    
    # Import data to pandas dataframe
    dat = ascii.read(cfile,header_start=c_column_line,data_start=c_header,delimiter=c_delimiter,exclude_names='N/A')
    datain = pd.DataFrame(dat.as_array())
    
    # Make TIMESTAMP the index
    datain['TIMESTAMP'] = datain['TIMESTAMP'].astype('datetime64[ns]')
    datain = datain.set_index('TIMESTAMP')
    
    # Set time zone
    datain.index = datain.index.tz_localize(tz_in)
    
    # Import header info
    headerinfo = pd.read_csv(cfile,nrows=2,skiprows=1)
    
    # Convert to xray
    ds = xr.Dataset.from_dataframe(datain)
    
    # Rename time
    ds.rename({'TIMESTAMP':'time'},inplace=True)
    
    # Add new site dimension and make record
    ds = xr.concat([ds],'site',data_vars='all')
    #ds_site['site'] = csta_name

    # Add Coords
    print('need to get correct site location info, make these variables')
    #c_LAT xr.DataArray(39.0,['site',0])
    #ds = xr.concat({'LAT':('site',[39.0])})
    ds.coords['LAT'] = 39.0
    ds.coords['LON'] = -120.0
    #ds.coords['sta_name'] = csta_name
    
    # Add variable attributes (units and sample period)
    for cvar in ds.data_vars:
        ds.get(cvar).attrs['unit']   = headerinfo.loc[0,[cvar]].values[0]
        #ds.get(cvar).attrs['sample'] = headerinfo.loc[1,[cvar]].values[0]
        
    print(ds.dims)
    ds_ALL.append(ds)
    
    # Export to netcdf
    cfileout = os.path.join(dir_out,csta_name + '.nc')
    ds.to_netcdf(cfileout,format='netcdf4') #,encoding={'AirTemp_Avg': {'dtype': 'int16', 'scale_factor': 0.1, '_FillValue': -9999}})

Processing BNS
need to get correct site location info, make these variables
Frozen(SortedKeysDict({'site': 1, 'time': 50980}))
Processing CRN
need to get correct site location info, make these variables
Frozen(SortedKeysDict({'site': 1, 'time': 50980}))
Processing FRS
need to get correct site location info, make these variables
Frozen(SortedKeysDict({'site': 1, 'time': 50980}))
Processing PWL
need to get correct site location info, make these variables
Frozen(SortedKeysDict({'site': 1, 'time': 50980}))


In [8]:
#ds_combined = xr.concat(ds_ALL,dim='site')
#ds_combined = xr.merge(ds_ALL,dim='site')


In [9]:
#ds_merged = ds_ALL[0]
#ds_merged.merge(ds_ALL[1])

In [17]:
# Open and combine multiple files
#os.chdir(dir_out)
#ds_ALL = xr.open_mfdataset('*.nc',chunks=10,concat_dim='site')

In [18]:
import collections
d = collections.OrderedDict()
d['site1'] = ds_ALL[0]
d['site2'] = ds_ALL[0]

In [20]:
X = xr.concat(d)

C:\Users\new356\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: the `dim` argument to `concat` will be required in a future version of xarray; for now, setting it to the old default of 'concat_dim'
  if __name__ == '__main__':


TypeError: can only concatenate xarray Dataset and DataArray objects

In [ ]:
#ds2